<a href="https://colab.research.google.com/github/hailusong/colab-god-idclass/blob/master/god_idclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Train Google Object Detection to Detect ID BBox

In [0]:
DEFAULT_HOME='/content'

## Environment Preparation

Establish security session with Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()

Use Google Cloud SDK gsutil to download the data file

In [3]:
# Download the file.
YOUR_GCS_BUCKET='id-norm'
!gsutil cp gs://{YOUR_GCS_BUCKET}/generated.tar.gz /tmp/generated.tar.gz
!ls /tmp/*gz

Copying gs://id-norm/generated.tar.gz...
\ [1 files][129.7 MiB/129.7 MiB]                                                
Operation completed over 1 objects/129.7 MiB.                                    
/tmp/generated.tar.gz


Prepare the data file (unzip, untar)

In [12]:
![[ ! -f /tmp/generated.tar && -f /tmp/generated.tar.gz ]] && gunzip /tmp/generated.tar.gz
![[ ! -e ./generated && -f /tmp/generated.tar ]] && tar xf /tmp/generated.tar
!ls generated

bbox-train-non-id1.csv	bbox-valid-on-dl.csv	pnts-valid-non-id2.csv
bbox-train-non-id2.csv	bbox-valid-on-hc.csv	pnts-valid-non-id3.csv
bbox-train-non-id3.csv	pnts-train-non-id1.csv	pnts-valid-on-dl.csv
bbox-train-on-dl.csv	pnts-train-non-id2.csv	pnts-valid-on-hc.csv
bbox-train-on-hc.csv	pnts-train-non-id3.csv	Train
bbox-valid-non-id1.csv	pnts-train-on-dl.csv	Valid
bbox-valid-non-id2.csv	pnts-train-on-hc.csv
bbox-valid-non-id3.csv	pnts-valid-non-id1.csv


Verify Versions: TF, Python, IPython and prompt_toolkit (these two need to have compatible version), and protoc

In [13]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [55]:
!python -V
!ipython --version
!pip show prompt_toolkit
!protoc --version

Python 3.6.7
5.5.0
Name: prompt-toolkit
Version: 1.0.15
Summary: Library for building powerful interactive command lines in Python
Home-page: https://github.com/jonathanslenders/python-prompt-toolkit
Author: Jonathan Slenders
Author-email: UNKNOWN
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: wcwidth, six
Required-by: jupyter-console, ipython
libprotoc 3.0.0


## Install Google Object Detection API in Colab
Reference is https://colab.research.google.com/drive/1kHEQK2uk35xXZ_bzMUgLkoysJIWwznYr


### Downgrade prompt-toolkit to 1.0.15
Run this **ONLY** if the Installation not Working

In [0]:
!pip install 'prompt-toolkit==1.0.15'

### Google Object Detection API Installation

In [35]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
!git clone --quiet https://github.com/tensorflow/models.git ${DEFAULT_HOME}
!ls ${DEFAULT_HOME}

AUTHORS     CONTRIBUTING.md    LICENSE	 README.md  samples    WORKSPACE
CODEOWNERS  ISSUE_TEMPLATE.md  official  research   tutorials


In [38]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')
!pwd

/content/models/research


*From Wikipedia ...*: 

**protocol buffers** are a language-neutral, platform-neutral extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler. 

You define how you want your data to be structured once, then you can **use special generated source code to easily write and read your structured data to and from a variety of data streams and using a variety of languages**.

Remember **.proto defines structured data** and **protoc generates the source code** the serailize/de-serialize.

In [0]:
!protoc object_detection/protos/*.proto --python_out=.
# !ls object_detection/protos/*.proto
# !cat object_detection/protos/anchor_generator.proto
# !ls AnchorGenerator*

In [54]:
import sys
sys.path.append('{DEFAULT_HOME}/models/research')
sys.path.append('{DEFAULT_HOME}/models/research/slim')
!ls {DEFAULT_HOME}/models/research

'$'				    lm_commonsense
 a3c_blogpost			    lstm_object_detection
 adversarial_crypto		    marco
 adversarial_logit_pairing	    maskgan
 adversarial_text		    minigo
 adv_imagenet_models		    models
 astronet			    morph_net
 attention_ocr			    namignizer
 audioset			    neural_gpu
 autoaugment			    neural_programmer
 autoencoder			    next_frame_prediction
 brain_coder			    nst_blogpost
 cognitive_mapping_and_planning     object_detection
 cognitive_planning		    pcl_rl
 compression			    ptn
 cvt_text			    qa_kg
 deep_contextual_bandits	    README.md
 deeplab			    real_nvp
 deep_speech			    rebar
 delf				    resnet
 differential_privacy		    sentiment_analysis
 domain_adaptation		    seq2species
 efficient-hrl			    setup.py
 feelvos			    skip_thoughts
 fivo				    slim
 gan				    steve
 global_objectives		    street
 im2txt				    struct2depth
 inception			    swivel
 keypointnet			    syntaxnet
 learned_optimizer		    tcn
 learning_to_remember_rare_events   tensorrt
 lea

In [61]:
!pwd
!python ./object_detection/builders/model_builder_test.py

/content/models/research
Traceback (most recent call last):
  File "./object_detection/builders/model_builder_test.py", line 23, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'
